# Calculate regional sea ice concentration for CMIP6 historical realizations

### Author - Chris Wyburn-Powell, see the latest version on [github](https://github.com/chrisrwp/low-frequency-variability/blob/main/input_data/Regional_sea_ice_CMIP6.ipynb)


**Input:**
- Sea ice concentration (SIC) from variable `siconc` or `siconca`. Historical forcings are used for 1850-2014 for all CMIP6 GCMs which appear in the CVDP historical groups A,B,C
- Regridded areacello/areacella files classificed by the NSIDC MASIE regions for each model grid

**Output:**
- 1850-2014 raw regional sea ice area, concentration, thickness and volume, as well as pan-Arctic sea ice area and volume
- 1920-2014 SIA for each region, 2 year lowpass filter of the linearly detrended data.

In [1]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
import datetime
import os
import pickle
import glob
import re
import dask
print(datetime.datetime.now())

2023-01-24 14:37:24.837553


In [673]:
#the following dask workers are sufficient for processing around 100 members
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(cores    = 1,
                     memory   = '8GB',
                     queue    = 'casper',
                     walltime = '00:02:00',
                     project  = 'UCUB0084',
                    )

cluster.scale(16)
client = Client(cluster)
client

/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/dav/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40762 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://10.12.206.46:46173 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/cwpowell/proxy/40762/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Load useful information

In [2]:
CMIP6_info = xr.open_dataset(
    '/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
    +'CMIP6_modeling_center_members_doi.nc'
)

xr_new_time = xr.open_dataarray(
    '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
    +'datetime64_1850_2100_monthly.nc')

In [3]:
with open('/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
          +'CMIP6_SImon_siconc_paths.pickle', 'rb') as handle:
    siconc_paths = pickle.load(handle)
    
# with open('/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
#           +'CMIP6_SImon_sivol_sithick_paths.pickle', 'rb') as handle:
#     sivol_sithick_paths = pickle.load(handle)

with open('/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
          +'CMIP6_SImon_var_availibility.pickle', 'rb') as handle:
    var_availability = pickle.load(handle)

with open('/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
          +'CMIP6_areacello_paths.pickle', 'rb') as handle:
    areacello_paths = pickle.load(handle)
    
with open('/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
          +'CMIP6_areacello_lat_names.pickle', 'rb') as handle:
    lat_names = pickle.load(handle)
    
with open('/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
          +'CMIP6_x_y_names.pickle', 'rb') as handle:
    x_y_names = pickle.load(handle)

In [4]:
#get a list of the NEW GCMs to process only
with open('/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
          +'CMIP6_areacello_lat_names_old.pickle', 'rb') as handle:
    lat_names_OLD = pickle.load(handle)
    
with open('/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
          +'CMIP6_areacello_lat_names.pickle', 'rb') as handle:
    lat_names_NEW = pickle.load(handle)

lat_names_OLD = list(lat_names_OLD.keys())
lat_names_NEW = list(lat_names_NEW.keys())

new_GCMs = np.setdiff1d(np.ravel(lat_names_NEW), np.ravel(lat_names_OLD), 
                        assume_unique=False)
print(new_GCMs)

#don't process thickness for new GCMs, add this info to var_availability dict
for GCM in new_GCMs:
    var_availability[GCM] = 'neither'
    
#also for the GCMs with added members since CVDP historical A and B were produced
var_availability['ACCESS-CM2'] = 'neither'
var_availability['ACCESS-ESM1-5'] = 'neither'
var_availability['CMCC-CM2-SR5'] = 'neither'
var_availability['MIROC-ES2L'] = 'neither'
var_availability['CESM2-FV2'] = 'neither'
var_availability['EC-Earth3'] = 'neither'
var_availability['EC-Earth3-Veg'] = 'neither'
var_availability['MPI-ESM1-2-LR'] = 'neither'
var_availability['MPI-ESM-1-2-HAM'] = 'neither'
var_availability['MRI-ESM2-0'] = 'neither'
var_availability['GISS-E2-1-G'] = 'neither'
var_availability['GISS-E2-1-H'] = 'neither'
var_availability['HadGEM3-GC31-LL'] = 'neither'
var_availability['NorESM2-LM'] = 'neither'
var_availability['E3SM-1-0'] = 'neither'
var_availability['NESM3'] = 'neither'

['CESM2-LENS' 'CMCC-CM2-HR4' 'CMCC-ESM2' 'CanESM5-1' 'E3SM-2-0'
 'EC-Earth3-AerChem' 'EC-Earth3-CC' 'GISS-E2-1-G-CC' 'GISS-E2-2-G'
 'GISS-E2-2-H' 'GISS-E3-G' 'IPSL-CM5A2-INCA' 'IPSL-CM6A-LR-INCA'
 'MIROC-ES2H' 'UKESM1-1-LL']


## Define function for loading data

In [5]:
def load_member_CMIP6(model, ripf, output_var, chunk=False):
    '''
    Open a single member file of either sea ice concentration or thickness
    from glade using xarray.open_dataset
    
    Parameters
    ----------
    model: string
        The name of the model e.g. CanESM5
    ripf : string,
        r<>: realisation (i.e. ensemble member), i<>: initialisation method, 
        p<>: physics, f<>: forcing, e.g. r15i1p2f1
    output_var : string,
        Variable of concentration of volume e.g. 'siconc', 'sivol', 'sithick',
        N.B. either sivol or sithick will chose whichever of the 2 has the best
        availibility as previously found when defining sicol_sithick_paths
    chunk : bool, optional
        Open with dask chunks (auto chunked) if true, do not use dask is false
    
    Returns
    ----------
        xarray.DataSet object from glade sea ice output
    '''  
    
    if output_var in ['siconc']:
        paths = siconc_paths[model+'_'+ripf]
    elif output_var in ['sithick', 'sivol']:
        paths = sivol_sithick_paths[model+'_'+ripf]
    else:
        print('variable not availible')
        
    ########## use the file path to open the NetCDF file using xarray ##########    
    if type(paths) == list:
    
        if chunk:
            if len(paths) > 1:
                if model[0:2] == 'EC':
                    data = xr.open_mfdataset(paths, combine='nested', 
                                             concat_dim='time', 
                                             chunks='auto')
                else:
                    data = xr.open_mfdataset(paths, combine='nested', 
                                             concat_dim='time', 
                                             chunks={'time':198})
            elif len(paths) == 1:
                data = xr.open_mfdataset(paths[0], combine='nested', 
                                         concat_dim='time', chunks={'time':198})
            else:
                data = xr.open_dataset(paths, chunks={'time':198})
        
        else:
            if len(paths) > 1:
                data = xr.open_mfdataset(paths, combine='nested', 
                                         concat_dim='time')
            else:
                data = xr.open_dataset(paths[0])
            
    else: 
        print('no data availible for', model, ripf, output_var)
        data = np.nan
            
    return(data)

## Make the region masks and define location of areacello files

In [589]:
#make reduced areacello files for all models to only include 30-90N
for model_name in CMIP6_info['model'].values:
    try:
        areacello = xr.open_dataset(areacello_paths[model_name])
        
        try:
            areacello_30N = areacello['areacello'].where(
                areacello[lat_names[model_name]]>30)
            
        except KeyError:
            areacello = areacello.rename({'areacella':'areacello'})
            areacello_30N = areacello['areacello'].where(
                areacello[lat_names[model_name]]>30)
            
        areacello_30N.attrs = areacello.attrs.copy()
        areacello_30N.to_netcdf(
            '/glade/work/cwpowell/low-frequency-variability/'\
            +'raw_data/masie_masks/areacello_{}_30N.nc'.format(model_name))
    
    except AttributeError:
        print(model_name, 'no areacello to extract northern hemisphere data')

In [32]:
#make reduced areacello files for GISS-E2-1-G as uses areacella
model_name = 'GISS-E2-1-G'
areacella = xr.open_dataset(areacello_paths[model_name])
areacello = areacella.rename({'areacella':'areacello'})

areacello_30N = areacello['areacello'].where(
    areacello['lat']>30)

areacello_30N.attrs = areacella.attrs.copy()
areacello_30N.to_netcdf(
    '/glade/work/cwpowell/low-frequency-variability/'\
    +'raw_data/masie_masks/areacello_{}_30N.nc'.format(model_name))

**Regrid the MASIE region mask for each of the model grids, using nearest neighbor for the areacello files** <br>
**Use `regrid_MAISE_regions_to_areacello.sh`, with a single model exampled below:** <br>
`cdo remapnn,areacello_<model_name>_30N.nc masiemask_ims4km.nc masiemask_<model_name>.nc`

## Calculate the regional sea ice concentration and thickness with `dask`

In [6]:
def regional_calc_dask(model_name, mem, areacello_with_nan, region_mask,
                       start_yr_=False, end_yr_=False):
    '''
    Calculate regional SIA, SIC, SIV, SIT from glade files of concentration
    (siconc) and thickness (sithick) or derived from volume (sivol)
    from historical CMIP6 model runs. Regions based on NSIDC MASIE regions. 
    Also compute the pan-Arctic sea ice area and volume.

    Parameters
    ----------
    model_name: string
        The name of the model e.g. CanESM5
    mem: string
        Variant ID e.g. 'r1ip1f1' where the letters correspond to:
        r<>: realisation (i.e. ensemble member),
        i<>: initialisation method, p<>: physics, f<>: forcing
    areacello_with_nan: xarray dataarray
        areacello or areacella with latitudes below 30N set to np.nan
    region_mask: xarray dataarray
        dataarray with the same coordinates and dimensions as the sea ice 
        variable data but with values corresponding to region or np.nan for 
        outside of the regions domain

    Returns
    ----------
        xarray.Dataset with variables of:
        regional SIA, regional average SIC, regional SIV, regional average SIT,
        pan-Arctic SIA, pan-Arctic SIV
    '''  
    ################## determine if subset of years will be used ###############
    if start_yr_:
        start_yr = start_yr_
    else:
        start_yr = 1850
        
    if end_yr_:
        end_yr = end_yr_
    else:
        end_yr = 2014
    
    ########################### load the data files ############################
    #load the SIC and SIT data and select above 30N
    if type(siconc_paths[model_name+'_'+mem]) != list:
        no_siconc = True
        print(f'no data availible for {model_name} {mem} siconc')
    
    else:
        no_siconc = False
        SIC = load_member_CMIP6(model_name, mem, 'siconc', chunk=True)
        
        #remove concentration values below 30N and convert to fraction from %
        #only exceptions where siconc not availible
        if model_name in ['GISS-E2-1-G','GISS-E2-2-G']: 
            SIC = SIC['siconca'].where(
                    ~xr.ufuncs.isnan(areacella_GISS_E2_1_G), drop=False)/100
        elif model_name == 'CESM2-LENS':
            SIC = SIC['aice'].where(areacello_with_nan, drop=False)
        else:
            SIC = SIC['siconc'].where(areacello_with_nan, drop=False)/100
  
    #load sivol/sithick if sivol is not availible, or pass if neither available 
    no_sivol_sithick = False
    if var_availability[model_name] == 'sivol':
        #need to load sivol and divide by the grid cell area first
        SIV = load_member_CMIP6(model_name, mem, 'sivol', chunk=True)
        
        if type(SIV) == float:
            no_sivol_sithick = True
        else:
            if model_name == 'GISS-E2-1-H' or 'GISS-E2-1-G':
                SIT = SIV['sivol'].where(
                    ~xr.ufuncs.isnan(areacella_GISS_E2_1_G), drop=False)
            else:
                SIT = SIV['sivol'].where(areacello_with_nan, drop=False)
    
    elif var_availability[model_name] == 'sithick':
        #no need to do calculations before processing the data, just load files
        SIT = load_member_CMIP6(model_name, mem, 'sithick', chunk=True)
        
        if type(SIT) == float:
            no_sivol_sithick = True
        else:
            SIT = SIT['sithick'].where(areacello_with_nan, drop=False)
    
    else:
        no_sivol_sithick = True
    
    #replace time with datetime64 for consistency across models
    if no_siconc == False:
        SIC = SIC.sortby('time') #EC Earth models do not automatically sort
        if model_name == 'CESM2-LENS':
            SIC = SIC.sel(time=slice(str(start_yr)+'-02', str(end_yr+1)+'-01')) 
        else:
            SIC = SIC.sel(time=slice(str(start_yr), str(end_yr))) 
        SIC['time'] = xr_new_time.sel(time=slice(str(start_yr),str(end_yr)))
    
    if no_sivol_sithick == False:
        SIT = SIT.sortby('time')
        SIT = SIT.sel(time=slice(str(start_yr), str(end_yr))) 
        SIT['time'] = xr_new_time.sel(time=slice(str(start_yr),str(end_yr)))

    ################ compute SIA, SIV and average SIC and SIT ##################
    #make an empty dataarray with nans for missing data
    xr_new_time_nan = xr_new_time.sel(
        time=slice(str(start_yr), str(end_yr))).astype(int).where(
        xr_new_time.sel(time=slice(str(start_yr), str(end_yr))).astype(
            int)==0.01,0
        )*np.nan
    
    #only do the calculation where there is sea ice
    if no_siconc == False:
        SIC = SIC.where(SIC>0)
    
    if no_sivol_sithick == False:
        SIT = SIT.where(SIT>0)
    
    #calculate the pan-Arctic SIA and SIV
    if no_siconc == False:
        pan_Arctic_SIA = (SIC * areacello_with_nan).sum(
            x_y_names[model_name][0]).sum(x_y_names[model_name][1])
    else:
        pan_Arctic_SIA = xr_new_time_nan
        
    if no_sivol_sithick==False:
        if model_name == 'GISS-E2-1-H' or 'GISS-E2-1-G':
            pan_Arctic_SIV = (SIT * areacella_GISS_E2_1_G).sum(
                x_y_names[model_name][0]).sum(x_y_names[model_name][1])
        else:
            pan_Arctic_SIV = (SIT * areacello_with_nan).sum(
                x_y_names[model_name][0]).sum(x_y_names[model_name][1])
    else:
        pan_Arctic_SIV = xr_new_time_nan
    
    #calculate regional data
    SIA_regions = []
    SIC_regions_av = []
    SIV_regions = []
    SIT_regions_av = []

    for region_ in np.arange(1,17):
        area_region = areacello_with_nan.where(
                region_mask['regions']==region_).sum()
        
        if no_siconc == False:
            SIC_region = SIC.where((region_mask['regions']==region_))
            SIA_region = (SIC_region * areacello_with_nan).sum(
                x_y_names[model_name][0]).sum(x_y_names[model_name][1])
            
            SIA_regions.append(SIA_region)
            SIC_regions_av.append(SIA_region / area_region)
        else:
            SIA_regions.append(xr_new_time_nan)
            SIC_regions_av.append(xr_new_time_nan)
        
        if no_sivol_sithick == False:
            if model_name == 'GISS-E2-1-H' or 'GISS-E2-1-G':
                SIT_region = SIT.where(
                    (region_mask_GISS_E2_1_G['regions']==region_)).where(
                    ~xr.ufuncs.isnan(areacella_GISS_E2_1_G))

                SIV_region = (SIT_region * areacella_GISS_E2_1_G).sum(
                    x_y_names[model_name][0]).sum(x_y_names[model_name][1])
            else:
                SIT_region = SIT.where((region_mask['regions']==region_))

                SIV_region = (SIT_region * areacello_with_nan).sum(
                    x_y_names[model_name][0]).sum(x_y_names[model_name][1])

            SIV_regions.append(SIV_region)
            SIT_regions_av.append(SIV_region / area_region)
        
        else: #append nan values of the same shape as the SIC derived data
            SIV_regions.append(xr_new_time_nan)
            SIT_regions_av.append(xr_new_time_nan)
            
    
    ######################### concatenate all regions ##########################
    SIA_regions =  xr.concat((SIA_regions),dim='region')
    SIA_regions['region'] = np.arange(1,17)
    SIC_regions_av = xr.concat((SIC_regions_av),dim='region')
    SIC_regions_av['region'] = np.arange(1,17)

    SIV_regions = xr.concat((SIV_regions),dim='region')
    SIV_regions['region'] = np.arange(1,17)
    SIT_regions_av = xr.concat((SIT_regions_av),dim='region')
    SIT_regions_av['region'] = np.arange(1,17)
    
    final_dataset = xr.Dataset(
            {'regional_SIA':SIA_regions, 'regional_SIC':SIC_regions_av,
             'regional_SIV':SIV_regions, 'regional_SIT':SIT_regions_av,
             'Arctic_SIA':pan_Arctic_SIA, 'Arctic_SIV':pan_Arctic_SIV,
            }
        )
    
    if model_name not in ['GFDL-ESM4', 'MCM-UA-1-0', 'CESM2', 'CESM2-FV2',
                          'CESM2-WACCM', 'CESM2-LENS','CESM2-WACCM-FV2', 
                          'CIESM', 'GFDL-CM4', 'KIOST-ESM']:
        if np.logical_not(np.logical_and(no_siconc, no_sivol_sithick)):
            final_dataset = final_dataset.drop('type')
    
    return(final_dataset)

### Compute each member of all models using dask

In [49]:
#the following needs to be defined if running GISS-E2-<1-G,1-H,2-G>
#because these use areacello for siconc and areacella for sivol, the areacella 
#is the same shape as for GISS-E2-1-G, the GISS-E2-1-G areacella is required
#to be land masked as partial grid cells exist for siconca and sivol
region_mask_GISS_E2_1_G = xr.open_dataset(
    '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
    +'masie_masks/masiemask_GISS-E2-1-G.nc')

areacella_GISS_E2_1_G = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/'\
        +'raw_data/masie_masks/areacello_GISS-E2-1-G_30N.nc')

areacella_GISS_E2_1_G = areacella_GISS_E2_1_G['areacello']

#load the land variable mrsos as a land mask as sftlf etc. not available
mrsos = xr.open_dataset(
    '/glade/work/cwpowell/low-frequency-variability/raw_data/masie_masks/'\
    +'mrsos_Lmon_GISS-E2-2-G_historical_r1i1p1f1_gn_185001-187512.nc')

areacella_GISS_E2_1_G = areacella_GISS_E2_1_G.where(
    mrsos['mrsos'].isel(time=0).where(mrsos['mrsos'].isel(time=0)==0)==0)

In [7]:
# model_name = 'CESM2-LENS'
# areacello_ = xr.open_dataset(
#         '/glade/work/cwpowell/low-frequency-variability/'\
#         +'raw_data/masie_masks/areacello_{}_30N.nc'.format(model_name)
#     )
# areacello_with_nan = areacello_['areacello'].rename({'nlat':'nj', 'nlon':'ni'})

# region_mask = xr.open_dataset(
#         '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
#         +'masie_masks/masiemask_{}.nc'.format(model_name)
#     )

# region_mask = region_mask.rename({'nlat':'nj', 'nlon':'ni'})
# region_mask = region_mask.drop('lat_2').drop('lon_2')
# areacello_ = areacello_.rename({'nlat':'nj', 'nlon':'ni'})

In [17]:
# for mem_ in CMIP6_info['members'].sel(model=model_name).where(CMIP6_info['members'].sel(model=model_name)!='0.0', drop=True).values[61:]:
#     print(datetime.datetime.now(), mem_)
#     if len(siconc_paths[f'CESM2-LENS_{mem_}']) > 0:

#         output = regional_calc_dask('CESM2-LENS', mem_, areacello_with_nan, region_mask,
#                                start_yr_=False, end_yr_=False)

#         to_save = output.compute()

#         to_save.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
#                     +f'raw_data/regional_sea_ice_CMIP6/Regional_SIC_SIT_{model_name}_'\
#                     +f'{mem_}.nc')

In [7]:
for model_name in ['E3SM-1-0',]: 

    mem_list = CMIP6_info['members'].sel(model=model_name).where(
            CMIP6_info['members'].sel(model=model_name)!='0.0', drop=True).values

    print(mem_list)
    mem_list = ['r5i1p1f1']
    

    #load the region masks
    region_mask = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
        +'masie_masks/masiemask_{}.nc'.format(model_name)
    )

    #load the areacello file
    areacello_ = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/'\
        +'raw_data/masie_masks/areacello_{}_30N.nc'.format(model_name)
    )

    areacello_with_nan = areacello_['areacello']

    #uncomment for CESM2-based GCMs
    # region_mask = region_mask.rename({'nlat':'nj', 'nlon':'ni'})
    # region_mask = region_mask.drop('lat_2').drop('lon_2')
    # areacello_ = areacello_.rename({'nlat':'nj', 'nlon':'ni'})

    #uncomment if doing time batches due to computational constraints
#     start_years = [1850, 1875, 1900, 1925, 1950, 1975, 2000]
#     end_years =   [1874, 1899, 1924, 1949, 1974, 1999, 2014]

#     for start_i in range(len(start_years)):
#         print(datetime.datetime.now(), start_years[start_i])
    for mem_ in mem_list:
        print(datetime.datetime.now(), model_name, mem_)

        #make an array of xarray.Datasets with dask chunks and wait to calculate
        with dask.config.set(**{'array.slicing.split_large_chunks': True}):
            mem_data = regional_calc_dask(
                model_name, mem_, areacello_with_nan, region_mask)#,
                # start_yr_ = start_years[start_i], 
                # end_yr_ = end_years[start_i])

        mem_data = mem_data.expand_dims({'member':[mem_]})

        #only now do the calculations of a single member
        to_save = mem_data.compute()

        to_save.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
            +f'raw_data/regional_sea_ice_CMIP6/Regional_SIC_SIT_{model_name}_'\
            +f'{mem_}.nc')

        # to_save.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
        #     +f'raw_data/regional_sea_ice_CMIP6/Regional_SIC_SIT_{model_name}_'\
        #     +f'{mem_}_{start_years[start_i]}_{end_years[start_i]}.nc')

## Check all realizations and designate any 'bad members'

**No siconc but CVDP:**
- AWI-CM-1-1-MR ['r1i1p1f1','r2i1p1f1','r3i1p1f1','r4i1p1f1','r5i1p1f1']
- AWI-ESM-1-1-LR ['r1i1p1f1']
- E3SM-1-1 ['r5i1p1f1']
- GISS-E2-1-G-CC ['r1i1p1f1'] 
- GISS-E3-G ['r1i1p1f1'] 
- IITM-ESM  ['r1i1p1f1']
- IPSL-CM6A-LR ['r33i1p1f1']
- KACE-1-0-G ['r1i1p1f1','r2i1p1f1','r3i1p1f1']
- MCM-UA-1-0 ['r1i1p1f1' 'r1i1p1f2']
- MPI-ESM1-2-LR ['r30i1p1f1']
- TaiESM1 ['r2i1p1f1']

**Siconc file exists but contains zeros or nans:**
- CAS-ESM2-0 ['r1i1p1f1','r2i1p1f1','r3i1p1f1','r4i1p1f1'] #only data for some regions
- CNRM-CM6-1 ['r21i1p1f2','r22i1p1f2','r23i1p1f2','r24i1p1f2','r25i1p1f2','r26i1p1f2','r27i1p1f2','r28i1p1f2','r30i1p1f2'] #nans and r30i1p1f2 has no siconc file
- CNRM-ESM2-1 ['r10i1p1f2','r7i1p1f2','r8i1p1f2','r9i1p1f2'] #nans
- EC-Earth3-Veg ['r10i1p1f1', 'r12i1p1f1'] #missing a few years in 1800s, files not on ESGF
- UKESM1-0-LL ['r13i1p1f2, 'r14i1p1f2', 'r15i1p1f2'] #r15i1p1f2 missing siconc, others nan values

In [169]:
month_ = 10
region_ = 1

model_name =  'E3SM-1-1'
print(model_name)
mem_list = CMIP6_info['members'].sel(model=model_name).where(
            CMIP6_info['members'].sel(model=model_name)!='0.0', drop=True).values
print(mem_list)
print(len(mem_list))

for mem_ in mem_list:
    try:
        mem_data = xr.open_dataset(
            '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
            +f'regional_sea_ice_CMIP6/Regional_SIC_SIT_{model_name}_{mem_}.nc')
        
        #check the maximum values for each region
        print(mem_data['regional_SIC'].sel(region=[1,2,3,4,5,6,11]).isel(
            member=0).max('time').values)
        
        plt.plot(mem_data['regional_SIC'].isel(member=0).sel(
            time=mem_data['time.month']==month_).sel(region=region_))
        
        if np.isnan(np.max(mem_data['regional_SIC'])):
            print(mem_, 'contains nans')
    
    except FileNotFoundError:
        print(mem_)

## Detrend using 2 year lowpass filter

Standardize to have winter maximum = 1 for regions 1,2,3,4,5,11. With different land masks some GCMs have maximum values near 70 or 80% not 100% as would be expected for regions 1-5,11. Therefore make the maximum 100% for each member. For the other regions it is not a good assumption that the winter maximum concentration ~100%, therefore do not correct these regions.

In [104]:
def filt_lowpass(data_, sample_freq, cutoff, order, ax_n, detrend=False,
                 standard=False):
    '''
    Filter a time series using a lowpass Butterworth filter. 
    Uses scipy.signal.butter and scipy.signal.filtfilt
    
    Parameters
    ----------
    data_ : n dimensional xarray dataarray,
        Data to detrend and/or standardize, which can contain nans
    sample_freq: float,
        The sampling frequency of the input data, typically sample_freq=1 [year]
    cutoff: float,
        The fraction of the nyquist frequency (itself half the sampling 
        frequency). To filter with a 2-year lowpass filter with
        sample_freq=1 (year), cutoff=0.25
    order: int
        The order of the Butterworth filter, typically 4-6
    ax_n : int
        Which axis to do the filtering on (time)
    detrend: bool
        Whether to detrend the data with a linear trend
    standard: bool
        Whether to standardize the data after filtering

    Returns
    ----------
        numpy array of the same shape as the input data
    '''

    if detrend: #detrend the data first
        data_ = (data_ * 0) + signal.detrend(data=data_.fillna(0), axis=ax_n)

    b, a = signal.butter(order, cutoff, btype='lowpass') #low pass filter
    #apply the filter forward and backward along a given axis
    filtered = signal.filtfilt(b, a, data_, axis=ax_n) 

    filtered_xr = (data_ * 0) + filtered

    if standard: #standardize the data
        filtered_xr = (filtered_xr - filtered_xr.mean('time')) \
                      / filtered_xr.std('time')

    return(filtered_xr)

### Loop through each model and member, detrend, filter, then if not nan add to a single file

In [165]:
var_ = 'regional_SIC'

for model_name in CMIP6_info['model'].drop_sel(model=[
    'AWI-CM-1-1-MR','AWI-ESM-1-1-LR','CAS-ESM2-0','FGOALS-f3-L','FGOALS-g3',
    'GISS-E2-1-G-CC','GISS-E3-G','IITM-ESM','KACE-1-0-G','MCM-UA-1-0']).values:
    print(datetime.datetime.now(), model_name)
    
    try:

        mem_list = CMIP6_info['members'].sel(model=model_name).where(
        CMIP6_info['members'].sel(model=model_name)!='0.0', drop=True).values

        all_mem_filt = []
        for mem_ in mem_list:
            try:
                unfiltered = xr.open_dataset(
                    '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
                    +'regional_sea_ice_CMIP6/Regional_SIC_SIT_'\
                    +f'{model_name}_{mem_}.nc')
                
                if model_name == 'CESM2-LENS':
                    unfiltered = unfiltered*100
                    unfiltered = unfiltered.assign_coords(member=mem_)
                    unfiltered = unfiltered.expand_dims(dim='member')
            
                #correct regions 1,2,3,4,5,11 to max 100%, leave other regions
                mem_data_adj = (unfiltered[var_].sel(region=[1,2,3,4,5,11])
                    / unfiltered[var_].sel(region=[1,2,3,4,5,11]).max('time'))

                unfiltered_cor = xr.concat((unfiltered[var_].drop_sel(
                    region=[1,2,3,4,5,11]),mem_data_adj), dim='region'
                                          ).sortby('region')   
        
                if np.isnan(np.max(unfiltered[var_])):
                    print(model_name, mem_, 'CONTAINS NANS')
                else:
                    all_month_filtered = []
                    for month_ in np.arange(1,13):
                        #detrend on time dimension for a single month 1920-2014
                        filt_data = (unfiltered_cor.sel(
                                time=unfiltered_cor['time.month']==month_).sel(
                                time=slice('1920','2014')).sel(member=mem_) * 0
                            + filt_lowpass(unfiltered_cor.sel(
                                time=unfiltered_cor['time.month']==month_).sel(
                                time=slice('1920','2014')).sel(member=mem_), 
                                sample_freq=1, cutoff=0.25, order=5, ax_n=1, 
                                detrend=True, standard=False))

                        filt_data['time'] = np.arange(1920,2015)
                        filt_data = filt_data.rename({'time':'year'})
                        all_month_filtered.append(filt_data)

                    all_month_data = xr.concat((all_month_filtered), dim='month')
                    all_month_data['month'] = np.arange(1,13)

                    all_mem_filt.append(all_month_data)
                
            except (FileNotFoundError):
                print(model_name, mem_, 'SKIPPED MEMBER - no raw data')

        all_mem_data = xr.concat((all_mem_filt), dim='member')
        
        #save all non-nan members to a single NetCDF file
        model_xr = xr.Dataset({'SIC':all_mem_data})

        model_doi = CMIP6_info['doi'].sel(model=model_name).values

        model_xr.attrs = {
            'Description': '2 year lowpass filter of linearly detrended '\
                +'regional average sea ice concentration (SIC) in % for the '\
                +f'climate model {model_name} for each month for 1920-2014 '\
                +'with historical forcing. Regions as defined for NSIDC '\
                +'MASIE-NH Version 1, doi:10.7265/N5GT5K3K.',
            'Timestamp'  : str(datetime.datetime.utcnow().strftime(
                "%H:%M UTC %a %Y-%m-%d")),
            'Data source': 'CMIP6 model output for siconc or siconca from '\
                 +f'{model_name}, doi:{model_doi}.',
            'Analysis'   : 'https://github.com/chrisrwp/'\
                +'low-fequency-variability/input_data/'\
                +'Regional_sea_ice_CMIP6.ipynb'
        }
        
        model_xr.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
            +f'input_data/Regional_SIC_detrended_lowpass_filter_{model_name}'\
            +'_1920_2014.nc')
        
    except (FileNotFoundError):
        print(model_name, mem_, 'FAILED - no raw data')
    except (ValueError):
        print(model_name, 'FAILED - all members contain nan data')
    

2023-01-25 10:37:27.495372 ACCESS-CM2
2023-01-25 10:37:29.243239 ACCESS-ESM1-5
2023-01-25 10:37:35.880000 BCC-CSM2-MR
2023-01-25 10:37:36.370218 BCC-ESM1
2023-01-25 10:37:36.844538 CAMS-CSM1-0
2023-01-25 10:37:37.350667 CESM2
2023-01-25 10:37:39.352609 CESM2-FV2
CESM2-FV2 r1i2p2f1 SKIPPED MEMBER - no raw data
2023-01-25 10:37:39.917524 CESM2-LENS
CESM2-LENS 1011.001 SKIPPED MEMBER - no raw data
CESM2-LENS 1031.002 SKIPPED MEMBER - no raw data
CESM2-LENS 1051.003 SKIPPED MEMBER - no raw data
CESM2-LENS 1071.004 SKIPPED MEMBER - no raw data
CESM2-LENS 1091.005 SKIPPED MEMBER - no raw data
CESM2-LENS 1111.006 SKIPPED MEMBER - no raw data
CESM2-LENS 1131.007 SKIPPED MEMBER - no raw data
CESM2-LENS 1151.008 SKIPPED MEMBER - no raw data
CESM2-LENS 1171.009 SKIPPED MEMBER - no raw data
CESM2-LENS 1191.010 SKIPPED MEMBER - no raw data
CESM2-LENS 1231.011 SKIPPED MEMBER - no raw data
CESM2-LENS 1231.012 SKIPPED MEMBER - no raw data
CESM2-LENS 1231.013 SKIPPED MEMBER - no raw data
CESM2-LENS 123